# Computer Vision (CV) Model Efficiency
<br>

**1. Introduction**


**2. Specialized ML systems**
    - 2.1 MobileNets
    - 2.2 ProjectionNets
    - 2.3 ShuffleNet
    - 2.4 EffNet


**3. Model Optimization & Compression**
    - 3.1 Pruning
    - 3.2 Quantization
    - 3.3 Encoding
    - 3.4 Remove Bloatware
    - 3.5 Weight Sharing


**4. Knowledge Distillation**


**5. Acceleration**<br><br>




## 1. Introduction

Previously we have built models from scratch as well as we have been using pre-build models. Many of these models are quite sophisticated performing complex tasks but they tend to be bulky models which makes them harder to deploy especially on mobile devices or devices with limited capabilities.

One obvious solution if to deploy our model on the cloud and forget about the issue. This solution has several drowbacks including the constant dependency on a quality internet connection.

The other approach would be to reduce the model size while mantaining the accuracy achieved in the more complex model. In order to tackle this particular issue, there are several approaches that will allow us to reduce the response time of the model:

- Specialized ML systems
- Model Optimization & Compression
    - Pruning
    - Inference reduction by quantization.
- Knowledge Distillation.
- Acceleration


## 2. Specialized ML Frameworks

There are specialized architectures that have been especifically designed to address the resource bottlenecks on mobile devices and devices with limited resources. These architectures are conceived to optimize the numbero of operations, replacing the classical convolution with their own version:
- MobileNets: uses depthwise convolution followed by a pointwise convolution.
- ProjectionNets: jointly trains using two types of networks. A full trainer neural network (using existing architectures like Feed-forward NNs or LSTM RNNs) combined with a simpler "projection" network that leverages random projections to transform inputs or intermediate representations into bits.
- ShuffleNet: uses pointwise convolution in groups and to combine the features produced a shuffle layer is added.
- EffNet: uses spatial separable convolution (depthwise conv splitted along x and y with separable pooling between them).


Other possibility apart from the previous structures, is use **ML Kit**. One of the core ML Kit capabilities is an automatic model compression powered by **Learn2Compress**, which enables custom on-device deep learning models in **TensorFlow Lite** that run efficiently on mobile devices, without developers having to worry about optimizing for memory and speed.

Learn2Compress generalizes the learning framework introduced in previous frameworks like ProjectionNet and incorporates several state-of-the-art techniques for compressing neural network models. It takes as input a large pre-trained TensorFlow model provided by the user, performs training and optimization and automatically generates ready-to-use on-device models that are smaller in size, more memory-efficient, more power-efficient and faster at inference with minimal loss in accuracy.

It applies compresison techniques that include pruning, distillation and quantization.

<img src="./fig/Learn2Compress_01.png" alt="Quantization & Inference" width="500"/>


## 3. Model Optimization & Compression

### 3.1. Pruning 

Pruning reduces model size by **removing weights or operations** that are least useful for predictions (e.g. low-scoring weights). This can be very effective especially for models involving sparse inputs or outputs which can be reduced up to 2x in size while retaining 97% of the original prediction quality.

Because we are removing connections from the NN, the new model will have lower accuracy since the model was trained for the original connections. That is why the model is fine tuned after pruning to regain the accuracy. It is noted that fully connected layers and CNNs can usually go upto 90% sparsity without losing any accuracy.

### 3.2. Quantization

Quantization techniques are particularly effective when applied during training and can improve inference speed by reducing the number of bits used for model weights and activations. For example, using 8-bit fixed point representation instead of floats can speed up the model inference, reduce power and further reduce size by 4x. Quantization involves mainly clustering weights together or rounding them off.

Apache MXNet* delivered quantization approaches to improve performance and reduce the inference deployment costs. By using lower precision (INT8), there are two main beneficts:

1. The computation is accelerated by lower precision instruction, like VNNI (Intel instructions).
2. Memory bandwidth is saved and it enables better local cache as well as power savings.

Apache MXNet supports model quantization from float32 to signed INT8 (s8) or unsigned INT8 (u8). S8 is designed for general inference and u8 is specific for CNNs. For most CNNs, ReLU is used as the activation function so output activations are non-negative. Thus, the benefit of u8 is obvious — we can use one more bit for the data to achieve better accuracy.

<img src="./fig/Model_Quantization_and_Inference.png" alt="Quantization & Inference" width="700"/>

The INT8 inference pipeline includes two stages based on the trained FP32 models (JSON file of saved models and their parameters).

1. **Quantization with calibration** (offline stage). In this stage a small fraction of images from the validation dataset (3–5%) will be used to collect statistical information (e.x. naive min/max or optimal thresholds based on entropy theory) and defining scaling factors using symmetric quantization and execution profiles of each layer. The output of this stage is a calibrated model including quantized operators saved as a JSON file and a parameter file.
2. **INT8 Inference** (run-time stage). The quantized and calibrated model should be a pair of a JSON file and a parameters file which can be loaded and used for inference just like the original model, except with higher speed and less accuracy difference.

As with pruning, we need to fine tune the model after quantization. The important point here is that the property that was given to the weights while quantization should be maintained through the fine tuning too. That is why specific ways of fine tuning are used which are tailored to match the quantization method.

### 3.3 Encoding

Encoding takes quantization one step further. By applying Huffman encoding and other lossless compression techniques, the pruned and quantized weights can be size-optimized further, so that we can represent the most frequent weight values with a lower number of bits. The obvious downside is an overhead in translation.

### 3.4 Remove Bloatware

A lot of memory is used to create graphs as this is useful for speeding up the training processes, but it isn’t used for inference. Hence the part of the graph used exclusively for training can be pruned off which is the part of the graph that we are callling bloatware.

For TensorFlow, it’s recommended to convert model checkpoints to frozen inference graphs. This process automatically removes the memory-hungry bloatware. Graphs from model checkpoints that throw Resource Exhausted Error can sometimes be fit into memory when converted to a frozen inference graph.

### 3.5 Weight Sharing

To best illustrate weight sharing, consider the example given in this Deep Compression paper. Consider a 4x4 weight matrix. It has 16 32-bit float values. We require 512 bits (16 * 32) to represent the matrix.
Let us quantize the weight values to 4 levels, but let’s preserve their 32-bit nature. Now, the 4x4 weight matrix has only 4 unique values. The 4 unique values are stored in a separate (shared) memory space. We can give each of the 4 unique values a 2-bit address (Possible address values being 0, 1, 2, and 3).

We can reference the weight values by using the 2-bit addresses. Hence, we obtain a new 4x4 matrix with 2-bit addresses, with each location in the matrix referring to a location in the shared memory space. This method requires 160 bits (16 * 2 + 4 * 32) for the entire representation. We obtain a size reduction factor of 3.2.
Needless to say, this reduction in size comes with an increase in time complexity. However, the time to access shared memory would not be a severe time penalty.


## 4. Joint Training & Knowledge Distillation

Joint training and distillation approaches follow a **teacher-student** strategy, where we use a complex and bulky model (teacher) to train a small network (student) with minimal loss in accuracy. The objective is to have the small network to produce comparable results to the large model (state of the art model or ensemble model) that is extracting very high level features from the data to produce accurate predictions.

### 4.1 Teacher and Student methodology

We can look at this process from the point of view of a Teacher and Student methodology. We do not train the  distilled model (student) on the raw data but instead we train the student to mimic the output of the large network (teacher). This implies the usage of how the deeper network learns hierarchical abstractions of the features.

The transferring of the generalization ability to the small model can be done by the use of the class probabilities produced by the teacher model as **soft targets** for training the student model. During this transfer stage, we use the same training set or a separate “transfer” set as used for training the teacher model. If our teacher model is an ensemble of simpler models, we can use arithmetic or geometric mean of their individual predictive distributions as the soft targets. When the soft targets have high entropy, they provide much more information per training case than hard targets and much less variance in the gradient between training cases, so the student model can often be trained on much less data than the original teacher model while using a much higher learning rate.

<img src="./fig/Model_distillation_01.png" alt="Model Distillation" width="700"/>

Much of the information about the learned function resides in the ratios of very small probabilities in the soft targets. This is valuable information that defines a rich similarity structure over the data (i. e. it tells as how much a car looks like a motorbike or a truck) but it has very little influence on the cross-entropy cost function during the transfer stage because the probabilities are so close to zero.

For distilling the learned knowledge we use Logits (the inputs to the final softmax). Logits can be used for learning the small model and this can be done by minimizing the squared difference between the logits produced by the teacher model and the logits produced by the student model.


## 5. Acceleration

Other advanced features to accelerate the inference quantization include (see Apache MXNet framework):

- Quantized data loader
- Offline calibration
- Graph optimization

Common computation patterns like convolution + relu are fused in the quantized network by a **graph optimizer** obtaining a more compact and efficient quantized network than the original. See example of the ResNet 50 v1  below with the network changes before and after the fusion and quantization.

<img src="./fig/ResNet50_quantization.png" alt="ResNet 50 Quantization" width="700"/>

### 5.1 Gradient Checkpoint

Frameworks such as TensorFlow consume a lot of memory for training because every node in the graph is saved into memory during a forward pass. This is required for calculating the gradient during backpropagation phase. The problem occurs as the models get deeper and deeper as the memory consumed in this way increases drastically. A neat solution is to recompute the values of the node when needed instead of saving them to memory. The drawback is that the computational cost increases significantly.

A good trade-off is to save only some nodes in memory while recomputing the others when needed. These saved nodes are called checkpoints. This drastically reduces deep neural network memory consumption.

This idea can be extented and we can recompute certain operations to save time. This is the case of Memory Efficient DenseNets.


### References:

- [1] [Custom on device ML models (Learn2Compress)](https://ai.googleblog.com/2018/05/custom-on-device-ml-models.html)
- [2] [MKL-DNN Quantization](https://mxnet.incubator.apache.org/api/python/docs/tutorials/performance/backend/mkldnn/mkldnn_quantization.html)
- [3] [Tensorflow Post Training Quantization](https://www.tensorflow.org/lite/performance/post_training_quantization)
- [4] [Model Quantization for production level Neural Network Interface](https://medium.com/apache-mxnet/model-quantization-for-production-level-neural-network-inference-f54462ebba05)
- [5] [Learning Efficient Object Detection Models with
Knowledge Distillation](https://papers.nips.cc/paper/6676-learning-efficient-object-detection-models-with-knowledge-distillation.pdf)
- [6] [Model Distillation](https://medium.com/neuralmachine/knowledge-distillation-dc241d7c2322)
- [7] [Training compact Deep Learning models for video classification - Araujo](http://openaccess.thecvf.com/content_ECCVW_2018/papers/11132/Araujo_Training_compact_deep_learning_models_for_video_classification_using_circulant_ECCVW_2018_paper.pdf)
- [8] [Improved Knowledge Distillation](https://arxiv.org/pdf/1902.03393.pdf)
- [9] [Dark knowledge of Neural Nets](https://towardsdatascience.com/knowledge-distillation-and-the-concept-of-dark-knowledge-8b7aed8014ac)
- [10] [Distilling the knowledge on a Neural Net](https://ai.google/research/pubs/pub44873)